# Data Science - Aufgabe 10 

## KNN Implementation

### Aufgabenstellung
Implement KNN by hand for just 2 dimensions with normalization.

This is easy because:

* funct: You normalize your data in another table
* funct: You code a simple euclid distance function
* funct: You take a point and calculate the distance to all points
* funct: You take the list from above and sort it
* funct: You aggregate by target variable
* funct: you take the max to determine the target class 

you are finished!
Note: This is the only chance to implement a machine learning algorithm by hand and hence learn something from the ground up!

### Impementierung

#### Szenario
Es liegen Daten zu 10 unterschiedlichen Lebensmittlen vor. Die erste Spalte stellte die Menge an Zucker in Gramm dar. Die zweite Spalte gibt die Menge an Süßstoffen (außer Zucker) in Milligramm an. Um einen Vergleichswert zu haben, gibt die dritte Spalte das Gesamtgewicht an. Die vierte Spalte klassifiziert das Lebensmittel als "Süßspeise" bzw. als "keine Süßspeise".

#### Datensatz

In [133]:
using Pkg
Pkg.add("DataFrames")
using DataFrames

  Resolving package versions...
   Updating `C:\Users\sebas\.julia\environments\v1.4\Project.toml`
 [no changes]
   Updating `C:\Users\sebas\.julia\environments\v1.4\Manifest.toml`
 [no changes]


In [134]:
column_names = ["suggar (g)", "sweetener (mg)", "weight (g)", "dessert"]
frame = DataFrame(column_names[1] => [20, 200, 0, 10, 0, 500, 0, 200, 75, 350], 
     column_names[2] => [0, 0, 0, 3, 2, 0, 35, 5, 22, 0], 
     column_names[3] => [800, 3000, 500, 1000, 700, 1200, 650, 2500, 1350, 800], 
     column_names[4] => [false, false, false, false, false, true, true, true, true, true])
println(frame)

10×4 DataFrame
│ Row │ suggar (g) │ sweetener (mg) │ weight (g) │ dessert │
│     │ Int64      │ Int64          │ Int64      │ Bool    │
├─────┼────────────┼────────────────┼────────────┼─────────┤
│ 1   │ 20         │ 0              │ 800        │ 0       │
│ 2   │ 200        │ 0              │ 3000       │ 0       │
│ 3   │ 0          │ 0              │ 500        │ 0       │
│ 4   │ 10         │ 3              │ 1000       │ 0       │
│ 5   │ 0          │ 2              │ 700        │ 0       │
│ 6   │ 500        │ 0              │ 1200       │ 1       │
│ 7   │ 0          │ 35             │ 650        │ 1       │
│ 8   │ 200        │ 5              │ 2500       │ 1       │
│ 9   │ 75         │ 22             │ 1350       │ 1       │
│ 10  │ 350        │ 0              │ 800        │ 1       │


#### Aufgabe 
Mit Hilfe des KNN Algorithmus soll untersucht werden, ob es sich bei dem Dateneintrag \[50, 20, 1800\] um eine Süßspeise handelt oder nicht.

#### Lösung

##### Features generieren
Zu Beginn werden die Gewichtsdaten von Zucker und Süßstoff durch das Gesamtgewicht der Speise geteilt, damit nur noch zwei relevante Features übrig bleiben. 

In [135]:
feature_column_names = ["suggar prop", "sweetener prop", "is dessert", "distance"]

feature_frame = DataFrame()
suggar_proportions = Float64[]
for row in eachrow(frame[:,[1,3]])
    suggar = row[1]
    weight = row[2]
    push!(suggar_proportions, suggar/weight)
end
feature_frame[!, feature_column_names[1]] = suggar_proportions

sweetener_proportions = Float64[]
for row in eachrow(frame[:,[2,3]])
    sweetener = row[1]
    weight = row[2]
    push!(sweetener_proportions, (sweetener/1000)/weight) #Hier beachten, dass die Süßstoff Einträge in Milligramm angegeben sind.
end
feature_frame[!, feature_column_names[2]] = sweetener_proportions
feature_frame[!, feature_column_names[3]] = copy(frame[!, column_names[4]])

println(feature_frame)

10×3 DataFrame
│ Row │ suggar prop │ sweetener prop │ is dessert │
│     │ Float64     │ Float64        │ Bool       │
├─────┼─────────────┼────────────────┼────────────┤
│ 1   │ 0.025       │ 0.0            │ 0          │
│ 2   │ 0.0666667   │ 0.0            │ 0          │
│ 3   │ 0.0         │ 0.0            │ 0          │
│ 4   │ 0.01        │ 3.0e-6         │ 0          │
│ 5   │ 0.0         │ 2.85714e-6     │ 0          │
│ 6   │ 0.416667    │ 0.0            │ 1          │
│ 7   │ 0.0         │ 5.38462e-5     │ 1          │
│ 8   │ 0.08        │ 2.0e-6         │ 1          │
│ 9   │ 0.0555556   │ 1.62963e-5     │ 1          │
│ 10  │ 0.4375      │ 0.0            │ 1          │


##### Spalten normalisieren
Wenige Milligramm Süßstoff entsprechen einige Gramm Zucker. Somit können diese Spalten nicht direkt miteinander verglichen werden. Darum werden die Spalten für den KNN Algorithmus normalisiert.
Zuvor wird der zu testende Datensatz der Tabelle hinzugefügt, damit dieser ebenfalls normalisiert wird

In [136]:
function normalize!(frame)
    for column in eachcol(frame)
        sorted_column = sort(column)
        min = sorted_column[1]
        max = sorted_column[end]
        delta = max - min;
        for row in eachrow(column)
            value = row[1]
            row[1] = (value - min) / delta
        end
    end
end

normalized_feature_frame = copy(feature_frame)
push!(normalized_feature_frame, [50/1800, 20/(1800*1000), false])
normalize!(normalized_feature_frame[!, [1,2]])
println(normalized_feature_frame)

11×3 DataFrame
│ Row │ suggar prop │ sweetener prop │ is dessert │
│     │ Float64     │ Float64        │ Bool       │
├─────┼─────────────┼────────────────┼────────────┤
│ 1   │ 0.0571429   │ 0.0            │ 0          │
│ 2   │ 0.152381    │ 0.0            │ 0          │
│ 3   │ 0.0         │ 0.0            │ 0          │
│ 4   │ 0.0228571   │ 0.0557143      │ 0          │
│ 5   │ 0.0         │ 0.0530612      │ 0          │
│ 6   │ 0.952381    │ 0.0            │ 1          │
│ 7   │ 0.0         │ 1.0            │ 1          │
│ 8   │ 0.182857    │ 0.0371429      │ 1          │
│ 9   │ 0.126984    │ 0.302646       │ 1          │
│ 10  │ 1.0         │ 0.0            │ 1          │
│ 11  │ 0.0634921   │ 0.206349       │ 0          │


##### Distanz berechnen
Nun wird für den KNN Algorithmus eine Distanz Funktion implementiert. Dieser berechnet die Distanz zwischen zwei Vektoren. Es wird beispielhaft die Distanz zwischen zwei Zeilen des "feature_frame" berechnet.

Für den Testeintrag wird die Distanz zu jedem der Trainigsdaten berechnet. Anschließend wird die Tabelle nach der Distanz sortiert.

In [137]:
function calc_distance(toVector, fromVector)::Float64
    if(length(toVector) != length(fromVector))
        throw(ArgumentError("The arrays have to be of the same length"))
    end
    sum = 0
    for i in 1:length(toVector)
        sum += (toVector[i]-fromVector[i])^2
    end
    return sqrt(sum)
end

from = feature_frame[4, :]
to = feature_frame[9, :]
from = [from[feature_column_names[1]], from[feature_column_names[2]]]
to = [to[feature_column_names[1]], to[feature_column_names[2]]]
println(from)
println(to)
println(calc_distance(from, to))

[0.01, 3.0e-6]
[0.05555555555555555, 1.6296296296296297e-5]
0.045555557495949965


In [138]:
distances = Float64[]
index = Int64[]
normalized_test_data = last(normalized_feature_frame, 1)
normalized_feature_frame_other = first(normalized_feature_frame, nrow(normalized_feature_frame)-1)
distances_frame = copy(normalized_feature_frame_other)

suggar = normalized_test_data[!,feature_column_names[1]][1]
sweetener = normalized_test_data[!,feature_column_names[2]][1]
to = [suggar,sweetener]

i = 1
for row in eachrow(normalized_feature_frame_other)
    suggar = row[feature_column_names[1]]
    sweetener = row[feature_column_names[2]]
    from = [suggar, sweetener]
    push!(distances, calc_distance(from, to))
    push!(index, i)
    i += 1
end

distances_frame[!, feature_column_names[4]] = distances
distances_frame[!, "Index"] = index
distances_frame_sorted = sort(distances_frame, feature_column_names[4])
println(normalized_test_data)
println(distances_frame_sorted)

1×3 DataFrame
│ Row │ suggar prop │ sweetener prop │ is dessert │
│     │ Float64     │ Float64        │ Bool       │
├─────┼─────────────┼────────────────┼────────────┤
│ 1   │ 0.0634921   │ 0.206349       │ 0          │
10×5 DataFrame
│ Row │ suggar prop │ sweetener prop │ is dessert │ distance │ Index │
│     │ Float64     │ Float64        │ Bool       │ Float64  │ Int64 │
├─────┼─────────────┼────────────────┼────────────┼──────────┼───────┤
│ 1   │ 0.126984    │ 0.302646       │ 1          │ 0.115344 │ 9     │
│ 2   │ 0.0228571   │ 0.0557143      │ 0          │ 0.156019 │ 4     │
│ 3   │ 0.0         │ 0.0530612      │ 0          │ 0.165917 │ 5     │
│ 4   │ 0.0571429   │ 0.0            │ 0          │ 0.206447 │ 1     │
│ 5   │ 0.182857    │ 0.0371429      │ 1          │ 0.207072 │ 8     │
│ 6   │ 0.0         │ 0.0            │ 0          │ 0.215896 │ 3     │
│ 7   │ 0.152381    │ 0.0            │ 0          │ 0.22468  │ 2     │
│ 8   │ 0.0         │ 1.0            │ 1          │ 0

##### Voting
Nun wird abgestimmt, ob die Testdaten zu einer Süßspeise gehören oder nicht. Dazu werden die obersten k Elemente der sortierten Liste betrachtet und die Klassifizierungen gezählt. Das Test-Datenset wird der Klasse zugeordnet, die am meisten 'Votes' hat. 

In [139]:
k = 3
true_count = 0
first_k_entries = first(distances_frame_sorted, k)
for entry in eachrow(first_k_entries)
    if(entry[feature_column_names[3]])
        true_count += 1
    end
end

println(true_count)
if (true_count > k/2)
    println("The test data is of a dessert")
else 
    println("The test data is not of a dessert")
end

1
The test data is not of a dessert


#### Resultat
Der Eintrag mit der kleinsten Distanz gehört zu einer Süßspeise. Dennoch klassifiziert der KNN-Algorithmus die Testdaten als "nicht Süßseise", da die folgenden beiden Einträge zu keiner Süßspeise gehören. Je nachdem wie k definiert wurd erhält man ein anderes Ergebnis. Bei einer geringen Datenmenge von 10 Zeilen, erscheint mir k = 3 gut gewält zu sein.